In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.NoiseModel import construct_noise_model_QuantinuumH1_1
from LogicalQ.Benchmarks import n_qubit_ghz_generation
from LogicalQ.Experiments import benchmark_noise, circuit_scaling_experiment, noise_scaling_experiment, qec_cycle_efficiency_experiment
from LogicalQ.Analysis import circuit_scaling_bar3d
from LogicalQ.Analysis import noise_model_scaling_bar
from LogicalQ.Analysis import qec_cycle_efficiency_bar

In [ ]:
# We use the Steane code to demonstrate the library
steane_stabilizer_tableau = [
  "XXXXIII",
  "IXXIXXI",
  "IIXXIXX",
  "ZZZZIII",
  "IZZIZZI",
  "IIZZIZZ",
]

# Circuit Scaling

In [ ]:
# Construct circuit factories as circuit inputs
physical_circuit_factory = n_qubit_ghz_generation

def log_n_qubit_ghz_generation(n_qubits=3, circuit_length=None, barriers=False):
    pqc = n_qubit_ghz_generation(n_qubits, circuit_length, barriers=barriers)
    lqc = LogicalCircuit.from_physical_circuit(pqc, label=(7,1,3), stabilizer_tableau=steane_stabilizer_tableau)
    lqc.measure_all(with_error_correction=False)
    return lqc

logical_circuit_factory = log_n_qubit_ghz_generation

# Construct noise model factories as noise model inputs
noise_model_factory = construct_noise_model_QuantinuumH1_1

In [ ]:
phys_data = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=64,
    shots=128,
    with_mp=True
)

In [ ]:
log_data = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=True
)

In [ ]:
circuit_scaling_bar3d(phys_data)

In [ ]:
circuit_scaling_bar3d(log_data)

# Noise Model Scaling

In [ ]:
# Sweep 3x3x3 = 27 combinations of noise values.
error_scan_keys = ["p1q", "p2q", "pr"]
error_scan_val_lists = [
    [1e-2, 1e-3, 1e-4],   # single-qubit depolarization
    [1e-2, 1e-3, 1e-4],   # two-qubit depolarization
    [2e-2, 1e-2, 5e-3]    # read-out flip
]

base_noise_model = construct_noise_model_QuantinuumH1_1()

In [ ]:
noise_data = noise_scaling_experiment(
    circuit_inputs = [physical_circuit_factory(n_qubits=3)],
    noise_model_inputs = base_noise_model,
    error_scan_keys = error_scan_keys,
    error_scan_val_lists = error_scan_val_lists,
    compute_exact = False,
    shots = 256
)

In [ ]:
noise_model_scaling_bar(noise_data, scan_keys=error_scan_keys, separate_plots=True)

# QEC Cycle Efficiency

In [ ]:
# Inject 0–4 cycles of Steane QEC and see how fidelity/num_cycle behaves.
# After n cycles, how much logical fidelity have we recovered per cycle?
config_scan_keys = ["cycles"]
config_scan_val_lists = [list(range(5))] 

# Build logical GHZ circuit (3 logical qubits here for demo)
lqc_single = log_n_qubit_ghz_generation(n_qubits=3)

# Concrete noise model (base Quantinuum H1-1)
base_noise_model = construct_noise_model_QuantinuumH1_1()

In [ ]:
qec_eff_data = qec_cycle_efficiency_experiment(
    circuit_inputs = [lqc_single],
    noise_model_input = base_noise_model,
    config_scan_keys = config_scan_keys,
    config_scan_val_lists = config_scan_val_lists,
    method = "density_matrix",
    shots = 256
)

In [ ]:
qec_cycle_efficiency_bar(qec_eff_data)